In [1]:
import math
import os
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import random
from joblib import Parallel, delayed

In [2]:
##parameter setting
trans_mat=np.array([[0.6,0.4],[0.8,0.2]])
trans_mat_new=np.array([[0.5,0.5],[0.7,0.3]])


In [3]:
#################################
###### move one step forward ####
#################################


def step(last_obs):
    # last_obs: last observation of state

    if last_obs == 1:
        a = np.random.binomial(1, p=trans_mat[0,1])
        r = np.random.normal(2, 1)  # reward at the same stage
        s_next = a + 1  # next state
    elif last_obs == 2:
        a = np.random.binomial(1, p=trans_mat[1,1])
        r = np.random.normal(-1, 1)
        s_next = a + 1

    return (a, r, s_next)

def step_new(last_obs):
    # last_obs: last observation of state
    if last_obs == 1:
        a = np.random.binomial(1, p=trans_mat_new[0,1])
        r = np.random.normal(2, 1)  # reward at the same stage
        s_next = a + 1  # next state
    elif last_obs == 2:
        a = np.random.binomial(1, p=trans_mat_new[1,1])
        r = np.random.normal(-1, 1)
        s_next = a + 1

    return (a, r, s_next)

In [4]:
#################################
#### generate one trajectory ####
#################################


def gen_traj(T, gam, seed=None, s_init=None):
    # seed: random seed
    # s_init: initial state
    # gam: discount
    # T: iterative number

    # initialize the state
    if seed is None and s_init is None:
        s = np.random.binomial(1, p=0.5) + 1
    elif seed is not None:
        np.random.seed(seed)
        s = np.random.binomial(1, p=0.5) + 1
    if s_init is not None:
        s = s_init

    s_traj = [s]
    a_traj = []
    r_traj = []

    ret = 0
    for i in range(T):
        a, r, s_next = step(s)
        s_traj.append(s_next)
        a_traj.append(a)
        r_traj.append(r)
        s = s_next  # update current S as S_next
        ret += r * gam**i

    ## output state, reward trajectory. return
    return [s_traj, a_traj, r_traj, ret]


def gen_traj_new(T, gam, seed=None, s_init=None):
    # seed: random seed
    # s_init: initial state
    # gam: discount
    # T: iterative number

    # initialize the state
    if seed is None and s_init is None:
        s = np.random.binomial(1, p=0.5) + 1
    elif seed is not None:
        np.random.seed(seed)
        s = np.random.binomial(1, p=0.5) + 1
    if s_init is not None:
        s = s_init

    s_traj = [s]
    a_traj = []
    r_traj = []

    ret = 0
    for i in range(T):
        a, r, s_next = step_new(s)
        s_traj.append(s_next)
        a_traj.append(a)
        r_traj.append(r)
        s = s_next  # update current S as S_next
        ret += r * gam**i

    ## output state, reward trajectory. return
    return [s_traj, a_traj, r_traj, ret]

In [5]:
#######################
#### generate data ####
#######################


def data_gen(N, T_obs, T, gam, seed=None, s_init=None):
    # N: number of trajectories
    # T_obs: observed stage numbers

    s_data = np.zeros((N, T_obs), dtype=int)
    a_data = np.zeros((N, T_obs), dtype=int)
    r_data = np.zeros((N, T_obs))
    ret_data = []

    for i in range(N):
        if seed is not None:
            seed += 1
        tmp = gen_traj(T, gam, seed, s_init)
        s_data[i] = tmp[0][0:T_obs]  # store the i-th state trajectory
        a_data[i] = tmp[1][0:T_obs]
        r_data[i] = tmp[2][0:T_obs]  # store the i-th reward trajectory
        ret_data.append(tmp[3])

    ## output observed state, reward trajectory and true return
    return [s_data, a_data ,r_data, ret_data]

def data_gen_new(N, T_obs, T, gam, seed=None, s_init=None):
    # N: number of trajectories
    # T_obs: observed stage numbers

    s_data = np.zeros((N, T_obs), dtype=int)
    a_data = np.zeros((N, T_obs), dtype=int)
    r_data = np.zeros((N, T_obs))
    ret_data = []

    for i in range(N):
        if seed is not None:
            seed += 1
        tmp = gen_traj_new(T, gam, seed, s_init)
        s_data[i] = tmp[0][0:T_obs]  # store the i-th state trajectory
        a_data[i] = tmp[1][0:T_obs]
        r_data[i] = tmp[2][0:T_obs]  # store the i-th reward trajectory
        ret_data.append(tmp[3])

    ## output observed state, reward trajectory and true return
    return [s_data, a_data ,r_data, ret_data]

In [7]:
####################################
### Quantile temperal difference ###
####################################


def QTD(state_traj,
        action_traj,
        reward_traj,
        state_card,
        action_card,
        quantile_num,
        gam,
        rate=None,
        init_val=None):
    # obs_traj: training data
    # state_card: cardinality of state space
    ## quantile_num: number of target conditional quantiles for each state
    # rate: learning rate
    # init_val: initial value

    if init_val == None:
        ret_con_quantile = np.zeros((state_card,action_card, quantile_num))
    elif init_value is not None:
        ret_con_quantile = init_value
    if rate == None:
        rate = 0.1

    #ret_con_quantile = np.zeros((state_card, quantile_num))

    n = np.shape(reward_traj)[0]  ## number of trajectories
    batch_num = np.shape(state_traj)[1] - 1  ## number of (x,r,x') tuples
    tau = [(2 * i + 1) / (2 * quantile_num) for i in range(quantile_num)]
    
    for k in range(n):
        for l in range(batch_num):
            for i in range(quantile_num):    
                s_current = state_traj[k, l]
                a_current = action_traj[k, l]
                r_current = reward_traj[k, l]
                s_next = state_traj[k, l + 1]
                a_next = action_traj[k, l + 1]
                #j = np.random.randint(0, quantile_num - 1)
                #ret_con_quantile[s_current - 1, i] += rate * tau[i] - rate * (
                #    r_current + gam * ret_con_quantile[s_next - 1, j] -
                #    ret_con_quantile[s_current - 1, i] < 0)
                ret_con_quantile[s_current - 1, a_current, i] +=  rate * np.mean(
                    [tau[i] - 1 * (r_current + gam * 
                                      ret_con_quantile[s_next - 1,a_next, j] - 
                                      ret_con_quantile[s_current - 1, a_current, i] < 0) 
                     for j in range(quantile_num)] 
                )

    ## output conditional quantiles
    return (ret_con_quantile)

def QTD_new(state_traj,
        action_traj,
        reward_traj,
        state_card,
        action_card,
        quantile_num,
        gam,
        seed,
        rate=None,
        init_val=None):
    # obs_traj: training data
    # state_card: cardinality of state space
    ## quantile_num: number of target conditional quantiles for each state
    # rate: learning rate
    # init_val: initial value

    if init_val == None:
        ret_con_quantile = np.zeros((state_card,action_card, quantile_num))
    elif init_value is not None:
        ret_con_quantile = init_value
    if rate == None:
        rate = 0.1

    #ret_con_quantile = np.zeros((state_card, quantile_num))
    
    n = np.shape(reward_traj)[0]  ## number of trajectories
    batch_num = np.shape(state_traj)[1] - 1  ## number of (x,a,r,x') tuples
    tau = [(2 * i + 1) / (2 * quantile_num) for i in range(quantile_num)]
    
    for k in range(n):
        for l in range(batch_num):
            seed+=1
            np.random.seed(seed)
            s_current = state_traj[k, l]
            a_current = action_traj[k, l]
            r_current = reward_traj[k, l]
            s_next = state_traj[k, l + 1]
            a_next = step_new(s_next)[0] #generate according to pi_a
            for i in range(quantile_num):    
               
                #j = np.random.randint(0, quantile_num - 1)
                #ret_con_quantile[s_current - 1, i] += rate * tau[i] - rate * (
                #    r_current + gam * ret_con_quantile[s_next - 1, j] -
                #    ret_con_quantile[s_current - 1, i] < 0)
                ret_con_quantile[s_current - 1, a_current, i] +=  rate * np.mean(
                    [tau[i] - 1 * (r_current + gam * 
                                      ret_con_quantile[s_next - 1,a_next, j] - 
                                      ret_con_quantile[s_current - 1, a_current, i] < 0)
                     for j in range(quantile_num)] 
                )

    ## output conditional quantiles
    return (ret_con_quantile)

In [8]:
def weighted_percentile(data, weights, perc):

    data = np.array(data)
    weights = np.array(weights)
    idx = np.argsort(data)
    data = data[idx] # sort data
    weights = weights[idx] # sort weights
    cdf = np.cumsum(weights) / np.sum(weights)
    count = np.sum([ cdf[i] <= perc for i in range(np.shape(cdf)[0]) ])
    #if output=infty return the maximum of V
    if data[count]==float('inf'):
        count-=1
    return(data[count])
    

In [9]:
def reg(data):
    n_tr=data[0].shape[0]
    horizon=data[0].shape[1]
    weight=np.ones(n_tr)
    weight_mat=trans_mat_new/trans_mat
    for i in range(n_tr):
        for j in range(horizon):
            
            weight[i]*=weight_mat[data[0][i,j]-1,data[1][i,j]]
    X=np.hstack((data[0][:,0].reshape(-1, 1),np.array(data[-1]).reshape(-1, 1)))
    y=weight.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    intercept=model.intercept_.item()
    coef=model.coef_.reshape(-1)
    return(intercept,coef)



In [10]:
def ratio_calc(intercept,coef,X):
    ratio= np.dot(X,coef)+intercept
    return ratio

In [11]:
def conf_set_f(data,intercept,coef,alp_sc,alp):
    x=data[0][:,0]
    y=np.array(data[-1])
    X=np.hstack((x.reshape(-1, 1),y.reshape(-1, 1)))
    n=np.shape(data[0])[0]
    quant_interval_lower=np.zeros(2)
    quant_interval_upper=np.zeros(2)
    ratio=np.dot(X,coef.reshape(-1,1))+intercept
    ratio = np.where(ratio < 0, 0, ratio) 
    conf_hi = np.zeros(2)
    conf_lo = np.zeros(2)
    ## lower and upper quanitles for each states 
    for i in range(2):
        quant_interval_lower[i]=np.percentile(y[x==i+1],alp_sc/2*100)
        quant_interval_upper[i]=np.percentile(y[x==i+1],(1-alp_sc/2)*100)

    score=np.zeros((n,2))  
    
    for i in range(n):
        score[i,0]=quant_interval_lower[x[i]-1]-y[i]
        score[i,1]=y[i]-quant_interval_upper[x[i]-1]
    for i in range(2):
        conf_lo[i] = quant_interval_lower[i]-weighted_percentile(score[:,0],ratio,1-alp/2)
        conf_hi[i] = quant_interval_upper[i]+weighted_percentile(score[:,1],ratio,1-alp/2)
    return [conf_lo,conf_hi]



In [16]:
def coverage(data_test,conf_lo,conf_hi):
    n=np.shape(data_test[0])[0]
    acc=0
    for i in range(n):
        if data_test[-1][i]>conf_lo[data_test[0][i]-1] and data_test[-1][i]<conf_hi[data_test[0][i]-1]:
            acc+=1
    return acc/n

In [19]:
## calculate V estimator based on QTD output
def q_hat_f(G_quan):
    return (np.mean(G_quan, axis=2))




In [20]:
def CQR_quantile(alp,G_quan):
    s=np.shape(G_quan)[0]
    quan_num=np.shape(G_quan)[2]
    res=np.zeros((s,np.shape(G_quan)[1]))
    for i in range(s):
        data_aug=np.hstack([G_quan[i,0],G_quan[i,1]])
        weight_aug=np.hstack([np.ones(quan_num)*trans_mat_new[i,0],np.ones(quan_num)*trans_mat_new[i,1]])
        res[i,0]=weighted_percentile(data_aug, weight_aug, alp/2)
        res[i,1]=weighted_percentile(data_aug, weight_aug, 1-alp/2)
    return (res)

In [21]:
def conf_set(s_traj, r_traj,G_quan,gam,step_forward,alp_sc,alp):
     
    cqr_quan=CQR_quantile(alp_sc,G_quan)
    n=np.shape(s_traj)[0]
    u = np.random.randint(0, np.shape(G_quan)[2], size=n)
    x=s_traj[:,0]
    y=np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0)+[gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] for i in range(n)]
    
    


    conf_lo=np.zeros(2)
    conf_hi=np.zeros(2)
    score=np.zeros((n,2))  
    
    for i in range(n):
        score[i,0]=cqr_quan[x[i]-1,0]-y[i]
        score[i,1]=y[i]-cqr_quan[x[i]-1,1]
    for i in range(2):
        conf_lo[i] = cqr_quan[i,0]-weighted_percentile(score[:,0],np.ones(n),1-alp/2)
        conf_hi[i] = cqr_quan[i,1]+weighted_percentile(score[:,1],np.ones(n),1-alp/2)
    return [conf_lo,conf_hi]

In [23]:
def aggregation(conf_lo,conf_hi,data_test,xi):
    PI_lower = np.zeros(n_te)
    PI_upper = np.zeros(n_te)
    covs = np.zeros(n_te)
    lens = np.zeros(n_te)
    s_te = data_test[0][:,0]
    ret_te = np.array(data_test[-1])
    B=np.shape(conf_lo)[0]
    conf_1_lo = conf_lo[:,0]
    conf_1_hi = conf_hi[:,0]
    conf_2_lo = conf_lo[:,1]
    conf_2_lo = conf_hi[:,1]
    for j in range(n_te):
        
        Gs = list(conf_lo[:,s_te[j]-1]) + list(conf_hi[:,s_te[j]-1])
        Ws = [1 for i in range(2 * B)]
        Ss = [1 for i in range(B)] + list(np.zeros(B))
        tol = B *(1-xi)
        ind = np.argsort(Gs)
        Gs = [Gs[i] for i in ind]  ## ordered YS
        Ws = [Ws[i] for i in ind]  ## pure 1
        Ss = [Ss[i] for i in ind]  ## left 1 right 0
        count = 0
        leftpoints = None
        rightpoints = None

        for i in range(2 * B):
            if Ss[i] == 1:
                count = count + Ws[i]
                if count > tol and count - Ws[i] <= tol:
                    leftpoints = Gs[i]
            elif Ss[i] == 0:
                if count > tol and count - Ws[i] <= tol:
                    rightpoints = Gs[i]
                count = count - Ws[i]
    
        PI_lower[j] = leftpoints
        PI_upper[j] = rightpoints
        covs[j] = ret_te[j] >= leftpoints and ret_te[j] <= rightpoints
        lens[j] = rightpoints - leftpoints
    cov=np.mean(covs)
    len=np.mean(lens)
    return cov, len

In [24]:
#calculate pi_a(a|x)
p1=(trans_mat_new[0,1]/trans_mat_new[0,0])*(trans_mat[0,0]/trans_mat[0,1])
a1=p1/(1+p1)
p2=(trans_mat_new[1,1]/trans_mat_new[1,0])*(trans_mat[1,0]/trans_mat[1,1])
a2=p2/(1+p2)
def step_a(last_obs):
    if last_obs == 1:
        a = np.random.binomial(1, p=a1)
    elif last_obs == 2:
        a = np.random.binomial(1, p=a2)

    return (a)
    
vec_step=np.vectorize(step_a)
mat=np.array([[1-a1,a1],[1-a2,a2]])
weight_mat=trans_mat_new/trans_mat
def weight_calculation_clip(s_vec, a_vec,clip):
    weight=1
    step=np.shape(s_vec)[0]-1
    for i in range(step):
        weight*=weight_mat[s_vec[i]-1,a_vec[i]]
    weight=max(min(clip[1],weight),clip[0])
    return(weight)

In [25]:
## replay buffer+propensity score ratio
def replay_buffer(s_traj, a_traj, r_traj,step_forward,ratio,clip):
    n = np.shape(s_traj)[0]
    p = np.shape(s_traj)[1] - step_forward 
    Mem_state = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action_a = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_reward = np.zeros((n*p,step_forward+1))
    idx_weight=[]
    for i in range(n):
        for j in range(p):
            Mem_state[(i*p+j),:] = s_traj[i,j:(j+step_forward+1)]
            Mem_action_a[(i*p+j),:] = vec_step(s_traj[i,j:(j+step_forward+1)])
            Mem_action[(i*p+j),:] = a_traj[i,j:(j+step_forward+1)]
            Mem_reward[(i*p+j),:] = r_traj[i,j:(j+step_forward+1)]
            idx_weight.append(
            weight_calculation_clip(Mem_state[(i*p+j), :], Mem_action[(i*p+j), :],clip) * ratio[Mem_state[(i*p+j), 0]-1]
            )

    total = np.array(idx_weight).sum()
    idx_weight_final=np.array(idx_weight)/total
    return([Mem_state,Mem_action,Mem_reward,idx_weight_final])

##ratio of frequency of s_0/s_rb
def rb(s_traj, a_traj, r_traj,step_forward):
    n = np.shape(s_traj)[0]
    p = np.shape(s_traj)[1] - step_forward 
    Mem_state = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action_a = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_reward = np.zeros((n*p,step_forward+1))
    for i in range(n):
        for j in range(p):
            Mem_state[(i*p+j),:] = s_traj[i,j:(j+step_forward+1)]
            Mem_action_a[(i*p+j),:] = vec_step(s_traj[i,j:(j+step_forward+1)])
            Mem_action[(i*p+j),:] = a_traj[i,j:(j+step_forward+1)]
            Mem_reward[(i*p+j),:] = r_traj[i,j:(j+step_forward+1)]
    ratio_1=n*p/(2*n*p-np.sum(Mem_state[:,0]))
    ratio_2=n*p/(np.sum(Mem_state[:,0])-n*p)
    return([ratio_1,ratio_2])



In [26]:
def scoring(s_traj, r_traj,step_forward,gam,G_quan,v_hat):
    # s_traj: state trajectory
    # r_traj: reward trajectory
    # step_forward: number of steps used in approximating return
    # gam: discount
    if np.shape(s_traj)[1]!=step_forward+1:
        print("length dismatch")
    if np.shape(s_traj)[0]!=np.shape(r_traj)[0]:
        print("height dismatch")
    
    quan_num = np.shape(G_quan)[2]
    n = np.shape(s_traj)[0]
    u = np.random.randint(0, quan_num - 1, size=n)
    sc = list(
        map(
            abs,
            np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] -
                v_hat[s_traj[i, 0] - 1] for i in range(n)
            ]))

    return (sc)

In [27]:
def scoring_CQR(s_traj, r_traj,step_forward,gam,G_quan,CQR_quan):
    if np.shape(s_traj)[1]!=step_forward+1:
        print("length dismatch")
    if np.shape(s_traj)[0]!=np.shape(r_traj)[0]:
        print("height dismatch")
    
    quan_num = np.shape(G_quan)[2]
    n = np.shape(s_traj)[0]
    u = np.random.randint(0, quan_num - 1, size=n)
    sc = list(
        map(
            max,
            zip(
            -np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                 
                CQR_quan[s_traj[i, 0] - 1,0]-gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] 
                for i in range(n)
            ],
                
            np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] -
                CQR_quan[s_traj[i, 0] - 1,1] for i in range(n)
            ]   
                
                
                
            )))

    return (sc)

## 重复实验500次

In [28]:
import itertools

In [29]:
def new_rb_res(data_train, data_test, gam, alp, alp_sc, step_forward, QTD_para,B,tau,seed,sample_size,clip):
    n_tr, n_te = np.shape(data_train[0])[0], np.shape(data_test[0])[0]
    s_te = data_test[0]
    a_init_te = data_test[1]
    ret_te = data_test[3]
    
    ## split training data
    idx_perm = np.random.permutation(list(range(0, n_tr)))
    idx_tr, idx_cal = [idx_perm[0:int(n_tr / 2)], idx_perm[int(n_tr / 2):n_tr]]
    s_train_fold = data_train[0][idx_tr,:]
    a_train_fold = data_train[1][idx_tr,:]
    r_train_fold = data_train[2][idx_tr,:]
    
    ## train return distribution using QTD
    G_quan = QTD_new(state_traj=s_train_fold,
                 action_traj=a_train_fold,
                 reward_traj=r_train_fold,
                 state_card=2,
                 action_card=2,
                 quantile_num=QTD_para[0],
                 gam=gam,
                 seed=seed,
                 rate=QTD_para[1],
                 init_val=None)
    Q_hat = q_hat_f(G_quan)
    v_hat = np.zeros(2)
    v_hat[0]=trans_mat_new[0,0]*Q_hat[0,0]+trans_mat_new[0,1]*Q_hat[0,1]
    v_hat[1]=trans_mat_new[1,0]*Q_hat[1,0]+trans_mat_new[1,1]*Q_hat[1,1]
    CQR_quan = CQR_quantile(alp,G_quan)

    
    ## replay buffer
    l = np.shape(step_forward)[0]
    if isinstance(tau, int) == False:
        m = np.shape(tau)[0]
    elif isinstance(tau, int) == True:
        m = 1
        
    PI_cov = np.zeros((m,l))
    PI_len = np.zeros((m,l))

            
        
    for k in range(l):
               ## density ratio
        ratio=rb(s_traj=data_train[0][idx_tr, :],
                    a_traj=data_train[1][idx_tr, :],
               r_traj=data_train[2][idx_tr, :],
                step_forward=step_forward[k])
        
        conf_lo_sets=np.zeros((B,2))
        conf_hi_sets=np.zeros((B,2))
        for i in range(B):
            
        #n_cal = np.random.choice(a=[j for j in range(np.shape(Mem[0])[0])], p=p,size=200)
            Mem = replay_buffer(s_traj=data_train[0][idx_cal, :],
                    a_traj=data_train[1][idx_cal, :],
                 r_traj=data_train[2][idx_cal, :],
                step_forward=step_forward[k],
                               ratio=ratio,
                               clip=clip)
            weight_is=Mem[-1]
            n_cal = np.random.choice(range(np.shape(weight_is)[0]),size=sample_size, p=weight_is)
            ## calculate nonconformity scores based on calibration set
            conf_sets = conf_set(s_traj=Mem[0][n_cal,], 
                     r_traj=Mem[2][n_cal,],
                     G_quan=G_quan,
                     gam=gam,
                     step_forward=step_forward[k],
                     alp_sc=alp_sc,
                     alp=alp)
            conf_lo_sets[i,:]=conf_sets[0]
            conf_hi_sets[i,:]=conf_sets[1]
        for z in range(m):
            print()
            PI_cov[z,k], PI_len[z,k]=aggregation(conf_lo_sets,conf_hi_sets,data_test,tau[z])


            
            
    return [PI_cov,PI_len]


In [31]:
def foffano(data_train, data_test, gam, alp, alp_sc):
    n_tr, n_te = np.shape(data_train[0])[0],  np.shape(data_test[0])[0]
    s_init_te = data_test[0]
    a_init_te = data_test[1] 
    ret_te = data_test[-1]

    intercept,coef=reg(data_train)
    
    conf_set=conf_set_f(data=data_train,
         intercept=intercept,
        coef=coef,
        alp_sc=alp_sc,
        alp=alp_sc
        )
    
    quan_PI_cov=coverage(data_test,conf_set[0],conf_set[1])
    quan_PI_len=np.mean(conf_set[1]-conf_set[0])
  
    return ([quan_PI_cov, quan_PI_len])

In [33]:
#Parallel Calculation
def run_single_experiment(i, n_tr, gam, T_obs, seed, n_te, T, QTD_para, B, alp, alp_sc, tau, step_forward,sample_size,clip):

    data_train = data_gen(N=n_tr,
                          T_obs=T_obs,
                          T=T,
                          gam=gam,
                          seed=seed + i,
                          s_init=None)


    data_test = data_gen_new(N=n_te,
                             T_obs=1,
                             T=T,
                             gam=gam,
                             seed=seed + i + 10000,
                             s_init=None)

    result = new_rb_res(data_train=data_train,
                        data_test=data_test,
                        gam=gam,
                        alp=alp,
                        alp_sc=alp_sc,
                        step_forward=step_forward,
                        QTD_para=QTD_para,
                        B=B,
                        tau=tau,
                        seed=seed + i,
                        sample_size=sample_size,
                       clip=clip) 
   
    return result  # return [PI_cov_e, PI_len_e]




In [34]:
# Parameter setting
rep = 50
n_tr = 300
gam = 0.8
T_obs = 20
seed = 2025
n_te = 310
T = 20
QTD_para = [20, 0.1]
B = 50
alp = 0.1
alp_sc = 0.1
tau = [0.5,0.6]
clip=np.array([0.2,5.0])
step_forward = [1, 2]
sample_size=100




results = Parallel(n_jobs=28, verbose=1)(
    delayed(run_single_experiment)(
        i, n_tr, gam, T_obs, seed, n_te, T, QTD_para, B, alp, alp_sc,tau, step_forward, sample_size,clip
    ) for i in range(rep)
)



[Parallel(n_jobs=28)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done  46 out of  50 | elapsed:  1.2min remaining:    5.9s
[Parallel(n_jobs=28)]: Done  50 out of  50 | elapsed:  1.2min finished


In [35]:
rb_new_cov_tau05 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau06 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau05 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau06 = np.zeros((rep, np.shape(step_forward)[0]))
for i in range(np.shape(results)[0]):
    rb_new_cov_tau05[i, :] = results [i][0][0]
    rb_new_cov_tau06[i, :] = results [i][0][1] 



    rb_new_len_tau05[i, :] = results [i][1][0]
    rb_new_len_tau06[i, :] = results [i][1][1]
   

In [36]:
import pandas as pd
print("new(tau=0.5): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau05[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau05[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("std-coverage probability: ")
print([np.std(rb_new_cov_tau05[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("std-average length: ")
print([np.std(rb_new_len_tau05[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])


new(tau=0.5): 
coverage probability: 
[0.9073548387096774, 0.9126451612903226]
average length: 
[7.671753291976457, 7.773955719173105]
std-coverage probability: 
[0.009763914535428448, 0.008723763996844056]
std-average length: 
[0.14746043746456008, 0.12768182000345749]


In [38]:

print("new(tau=0.6): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau06[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau06[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("std-coverage probability: ")
print([np.std(rb_new_cov_tau06[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])
print("std-average length: ")
print([np.std(rb_new_len_tau06[:,i]) for i in range(np.shape(rb_new_cov_tau05)[1])])

new(tau=0.6): 
coverage probability: 
[0.919032258064516, 0.9229032258064518]
average length: 
[7.9090860777166725, 8.022843496213346]
std-coverage probability: 
[0.00801273699895111, 0.0075030344191051625]
std-average length: 
[0.1636792811408967, 0.14754633476568152]


In [39]:
n_tr = 300
gam = 0.8
T_obs = 20
seed = 2025
n_te = 310
T = 20
rep = 100
QTD_para = [20, 0.1]
alp = 0.1
alp_sc=0.1
res_quan_500_qnum10 = np.zeros((rep, 2))

# Parallel Calculation
def process_iteration(i,n_tr, gam, T_obs, seed, n_te, T, alp,alp_sc):
 
    data_train = data_gen(N=n_tr,
                         T_obs=T_obs,
                         T=T,
                         gam=gam,
                         seed=seed+i,
                         s_init=None)
    

    data_test = data_gen_new(N=n_te,
                            T_obs=1,
                            T=T,
                            gam=gam,
                            seed=seed + i + 10000,
                            s_init=None)
    
    quan_PI_res1 = foffano(data_train=data_train,
                                      data_test=data_test, 
                                      gam=gam, 
                                      alp=alp,
                                      alp_sc=alp_sc
                                      )
  
    print(f"test num: {i}")
    print("quantile region: ")
    print(f"cov: {quan_PI_res1[0]} | length: {quan_PI_res1[1]}")
    
    return quan_PI_res1



results_qr = Parallel(n_jobs=30)(delayed(process_iteration)(i, n_tr, gam, T_obs, seed, n_te, T, alp,alp_sc) for i in range(rep))

# restore data
for i in range(rep):
    res_quan_500_qnum10[i, :] = results_qr[i]

In [40]:
res_quan_500_qnum10

array([[0.8483871 , 6.36261743],
       [0.84516129, 6.35949009],
       [0.8483871 , 6.35949009],
       [0.8483871 , 6.3532521 ],
       [0.84193548, 6.31423382],
       [0.84193548, 6.31454911],
       [0.83870968, 6.3095209 ],
       [0.83870968, 6.31454911],
       [0.83870968, 6.3095209 ],
       [0.83870968, 6.3095209 ],
       [0.83870968, 6.31432095],
       [0.83870968, 6.31432095],
       [0.8516129 , 6.52801176],
       [0.8483871 , 6.55124495],
       [0.8483871 , 6.44806787],
       [0.84516129, 6.44806787],
       [0.84193548, 6.38430021],
       [0.84516129, 6.39565194],
       [0.84516129, 6.44661944],
       [0.84516129, 6.41120531],
       [0.84516129, 6.43106607],
       [0.84516129, 6.42214562],
       [0.84516129, 6.42214562],
       [0.84516129, 6.42214562],
       [0.84516129, 6.43106607],
       [0.84516129, 6.43106607],
       [0.84516129, 6.42214562],
       [0.8483871 , 6.43308594],
       [0.8483871 , 6.43308594],
       [0.84516129, 6.44402626],
       [0.

In [41]:
print("quantile region: ")
print("coverage probability: ", np.mean(res_quan_500_qnum10[:, 0]),
      "|  average length: ", np.mean(res_quan_500_qnum10[:, 1]),
      "|  std-cov: ", np.std(res_quan_500_qnum10[:, 0]),
      "|  std-len: ", np.std(res_quan_500_qnum10[:, 1])) 
#print([ np.mean(rb_new_cov_tau05_e[:,i]) for i in range(np.shape(rb_new_cov_tau05_e)[1])])
#print([ np.mean(rb_new_len_tau05_e[:,i]) for i in range(np.shape(rb_new_cov_tau05_e)[1])])

quantile region: 
coverage probability:  0.8539999999999999 |  average length:  6.510429285054148 |  std-cov:  0.009630207417348005 |  std-len:  0.11375181024677512
